# Tutorial: Basic visualizations using the Druid API

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->
  
This tutorial introduces basic visualization options you can use with the Druid API.

It focuses on two Python modules to accomplish visualization tasks: [pandas](https://pandas.pydata.org/) and [Bokeh](https://bokeh.org/).
This tutorial builds on [Learn the basics of the Druid API](api-tutorial.ipynb).

## Table of contents

- [Prerequisites](#Prerequisites)
- [Display data with pandas](#Display-data-with-pandas)
- [Display data with a bar graph](#Display-data-with-a-bar-graph)
- [Display data with a line graph](#Display-data-with-a-line-graph)
- [Next steps](#Next-steps)

For the best experience, use JupyterLab so that you can always access the table of contents.

## Prerequisites

If you haven't already, you'll need install the Requests library for Python before you start. For example:

```bash
pip3 install requests
```

Additionally, install the pandas and Bokeh libraries. For example:
```bash
pip3 install pandas
pip3 install bokeh
```

Next, you'll need a Druid cluster with data. This tutorial uses the `wikipedia_api` datasource from [Learn the basics of the Druid API](api-tutorial.ipynb).


Finally, you'll need either JupyterLab (recommended) or Jupyter Notebook. Both the quickstart Druid cluster and Jupyter are deployed at `localhost:8888` by default, so you'll 
need to change the port for Jupyter. To do so, stop Jupyter and start it again with the `port` parameter included. For example, you can use the following command to start Jupyter on port `3001`:

```bash
# If you're using JupyterLab
jupyter lab --port 3001
# If you're using Jupyter notebook
jupyter notebook --port 3001 
```

To start this tutorial, run the next cell. It imports the Python packages you'll need and defines the Druid host where the Druid Router service listens. The quickstart deployment configures the to listen on port `8888` by default, so you'll be making API calls against `http://localhost:8888`. 

In [1]:
import requests, json
import druidapi
import pandas as pd
from bokeh.palettes import Spectral10
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

# Instantiate the Druid API Rest client
# and a SQL client
druid = druidapi.client("http://localhost:8888")
sql_client = druid.sql()

# Datasource from the Druid API tutorial
dataSourceName = "wikipedia_api"


## Display data with pandas

By default, when you query Druid using the API, Druid returns the results as JSON. The JSON output is great for programmatic operations, but it is not easy to scan the data visually. This section shows you how to use the Python pandas module to transform JSON query results to tabular format. 

The following uses the Druid API SQL client to select the top 10 channels by the number of additions and represent the output using JSON as it comes from Druid.

In [2]:
sql = f'''
SELECT channel,
SUM(added) AS additions
FROM {dataSourceName}
GROUP BY channel
ORDER BY additions DESC LIMIT 10
'''

# Run the SQL query
# Return a JSON object with a list of rows
query_result = json.dumps(sql_client.sql_query(sql).rows())
query_result

'[{"channel": "#ceb.wikipedia", "additions": 3280235}, {"channel": "#sv.wikipedia", "additions": 2748599}, {"channel": "#en.wikipedia", "additions": 2068620}, {"channel": "#fr.wikipedia", "additions": 498994}, {"channel": "#ru.wikipedia", "additions": 403344}, {"channel": "#de.wikipedia", "additions": 380768}, {"channel": "#it.wikipedia", "additions": 267957}, {"channel": "#es.wikipedia", "additions": 225090}, {"channel": "#pt.wikipedia", "additions": 197268}, {"channel": "#nl.wikipedia", "additions": 135426}]'

The Druid API SQL client contains some formatting if you invoke the `style` and use the `show` method. For example:

In [3]:
# Invoke the Druid API styles
druidapi.styles()

# Run the query with the Druid API SQL client
sql_client.show(sql)

channel,additions
#ceb.wikipedia,3280235
#sv.wikipedia,2748599
#en.wikipedia,2068620
#fr.wikipedia,498994
#ru.wikipedia,403344
#de.wikipedia,380768
#it.wikipedia,267957
#es.wikipedia,225090
#pt.wikipedia,197268
#nl.wikipedia,135426


You can use the pandas library [`read_json`](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html) method to load JSON results from the Druid API into a pandas DataFrame.
Pandas enables you to display the results in a tabular format.
The `orient` parameter sets pandas to accept data as a list of records in the format: `[{column: value, ...}, {column: value, ...}]`.

In [4]:
additions_pd = pd.read_json(query_result, orient='records')

additions_pd

,channel,additions
0,#ceb.wikipedia,3280235
1,#sv.wikipedia,2748599
2,#en.wikipedia,2068620
3,#fr.wikipedia,498994
4,#ru.wikipedia,403344
5,#de.wikipedia,380768
6,#it.wikipedia,267957
7,#es.wikipedia,225090
8,#pt.wikipedia,197268
9,#nl.wikipedia,135426


The basis for the Bokeh examples in this tutorial are DataFrames. Another benefit of using a DataFrame is that you can access all the pandas `DataFrame` object methods. For example, `DataFrame.max()`:

In [ ]:
additions_pd.max()

Check out the [pandas docs](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html) for more ideas about how to use pandas and DataFrames 
with your Druid data.

## Display data with a bar graph

Tabular format is OK for scanning data; however, you can use visualization and plotting tools that work with Jupyter Notebooks to visualize data as plots or graphs. This section uses [Bokeh](https://bokeh.org/) to illustrate some basic plots using Druid data.

In this section, you create a simple bar chart showing the channels with the most additions during the time range.

First, call `output_notebook()` to set Bokeh to output plots inline within the notebook. This lets you view your plot inline. You can also use Bokeh's `output_file` function to write your plot to an HTML file. If you want to experiment with `output_file`, add it to the list of imports. For example:
```
from bokeh.io import output_notebook, show, output_file
```

In [5]:
output_notebook()

Loading BokehJS ...

There are several ways to use Bokeh with a DataFrame. In this case, make a list of channels to serve as the x-axis of our plot. For the y-axis,  divide the total additions by 100000 for ease of display.

In [6]:
channels = additions_pd.channel.to_list()
total_additions = [x / 100000 for x in additions_pd.additions.to_list()]

In [7]:
channels

['#ceb.wikipedia',
 '#sv.wikipedia',
 '#en.wikipedia',
 '#fr.wikipedia',
 '#ru.wikipedia',
 '#de.wikipedia',
 '#it.wikipedia',
 '#es.wikipedia',
 '#pt.wikipedia',
 '#nl.wikipedia']

In [8]:
total_additions

[32.80235,
 27.48599,
 20.6862,
 4.98994,
 4.03344,
 3.80768,
 2.67957,
 2.2509,
 1.97268,
 1.35426]

Next, initialize the Bokeh plot (figure) with some basic configurations.

In [9]:
 # Create a new plot with a title. Set the size in pixels
bar_plot = figure(height=500, width=750, x_range=channels,  title="Additions by channel",
           toolbar_location=None)

Now, configure the renderer for the vertical bars on the plot:
- Set the x-axis to `channels`, the list of channels.
- Set the `top` coordinate that determines the bar height to `total_additions`.
- For a splash of color, set the `color` to `Spectral10`. 

Note that palettes in Bokeh are lists of colors. Bokeh expects the list length to equal the list length of the data dimensions -- in this case 10 colors.

See the Bokeh docs for more information on [vertical bars](https://docs.bokeh.org/en/latest/docs/reference/plotting/figure.html#bokeh.plotting.figure.vbar) and [palettes](https://docs.bokeh.org/en/latest/docs/reference/palettes.html). Note the expected output for the cell: `GlyphRenderer(id = 'p1054', …)`

In [10]:
bar_plot.vbar(x=channels, top=total_additions, width=0.5, color=Spectral10)

GlyphRenderer(id='p1054', ...)

Leave off the x-axis grid lines for this plot.

In [11]:
bar_plot.xgrid.grid_line_color = None

Now, configure the y-axis:
 - Set the minimum value to `0`.
 - Set the visible range to `(0,40)`.

In [12]:
bar_plot.y_range.start = 0
y_range=(0, 40)

Finally, display your plot with the `show()` method.

In [13]:
show(bar_plot)

## Display data with a line graph

In this section, you'll create a line graph that compares the following, per channel:

* Total additions
* Number of unique editors
* Number of bot edits

First, change the query to include the editors and robots and load the results into a new pandas object.

In [14]:
sql = f'''
        SELECT channel, SUM(added) AS additions,
        COUNT (DISTINCT user) as editors,
        SUM(CASE WHEN isRobot='true' THEN 1 ELSE 0 END) AS robots
        FROM {dataSourceName}
        GROUP BY channel ORDER BY additions DESC LIMIT 10
        '''


# Submit the request and get a list of JSON objects.
query_result = json.dumps(sql_client.sql_query(sql).rows())

# Add the results to a pandas DataFrame.
editors_pd = pd.read_json(query_result, orient='records')

editors_pd

,channel,additions,editors,robots
0,#ceb.wikipedia,3280235,3,1808
1,#sv.wikipedia,2748599,74,1744
2,#en.wikipedia,2068620,3032,536
3,#fr.wikipedia,498994,537,180
4,#ru.wikipedia,403344,522,66
5,#de.wikipedia,380768,675,186
6,#it.wikipedia,267957,328,422
7,#es.wikipedia,225090,439,50
8,#pt.wikipedia,197268,205,13
9,#nl.wikipedia,135426,153,11


Set up values for your plots:
- Leave channels as the basis for the x-axis.
- Change the resolution for additions to 1000, so it will fit nicely on our plot with the editors and the robots values.
- Create lists of values for editors and robots.

In [15]:
channels = editors_pd.channel.to_list()
total_additions = [x / 1000 for x in editors_pd.additions.to_list()]
editors = editors_pd.editors.to_list()
robots = editors_pd.robots.to_list()

Next, create a plot the same as before, but this time, leave in the Bokeh tools so you can try those out at the end.

In [16]:
# Create a plot.
line_plot = figure(x_range=channels, height=500, width=750, title="Editors vs robots")

Change the scale to accommodate the current data set.

In [17]:
line_plot.y_range.start = 0
y_range=(0, 4000)

Next, add data lines onto the plot:
- For the line color, you only need one color per line. To keep with the Spectral palette, set the index (0-9) for the color you want.
- Bokeh adds a legend for the lines. The legend label identifies each line in the legend.
- The `line_width` and `line_dash` properties control the line appearance.

See the [Bokeh docs](https://docs.bokeh.org/en/latest/docs/reference/plotting/figure.html#bokeh.plotting.figure.line) for more information about the line figures.

In [18]:
# Add renderers for the lines
addition_line = line_plot.line(channels, total_additions, line_color=Spectral10[0], legend_label="Additions x 1000",line_width=3, line_dash='solid')
deletion_line = line_plot.line(channels, editors, line_color=Spectral10[3], legend_label="Editors", line_width=3, line_dash='dashed')
robots_line = line_plot.line(channels, robots, line_color=Spectral10[7], legend_label="Robots", line_width=3, line_dash='dotted')

Finally, display the plot.

In [19]:
show(line_plot)

The graph shows some interesting data: channels with more bots editing topics tend to have fewer users editing the topics. Those are some hard-working bots!

Notice the toolbar to the right of the plot. You can use it to focus and navigate around the plot.

## Next steps

This tutorial covers the absolute basics for visualization with Apache Druid, pandas, and Bokeh. See the following topics to learn more about the various visualizations you can build:

- [Druid SQL API](https://druid.apache.org/docs/latest/querying/sql-api.html)
- [pandas](https://pandas.pydata.org/)
- [Bokeh](https://bokeh.org/)





